In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import time
from sklearn import svm
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
train_data = []  # 16659
test_data = []  # 810'''读取文件'''
print('Reading files...')
print('\n')
train_f = open("train.txt", encoding='utf-8')
for line in train_f:
    train_data.append(line)
train_f.close()

test_f = open("test.txt", encoding='utf-8')
for line in test_f:
    test_data.append(line)
test_f.close()
print(train_data[0])
print(test_data[0])
print('\n')
print('Files get!...')

In [ ]:
'''去掉开头序号-> [[句子(单词间以,为分割)], 0/1]'''
print('Formatting...')
print('\n')
train_set = []
for unit in train_data:
    temp = unit.split(' ')
    if temp[1].isdigit():
        # 判断第一个字符是否为数字，如果是则去掉，如果不是则取句子
        temp = temp[3:-1]
        # 当前temp中的最后一部分为"\t.\n","最后一个引号" 为了保证格式规范，先去掉，最后在取corpus的时候加上句号即可
        # training_set.append(([temp, unit[-3:-2]]))
        train_set.append([' '.join(temp), unit[-3:-2]])
        # unit[-3:-2]为当前句子的label
    else:
        # temp[1]为引号，应去掉
        temp = temp[1:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        train_set.append([' '.join(temp), unit[-3:-2]])

test_set = []
for unit in test_data:
    temp = unit.split(' ')
    if temp[1].isdigit():
        temp = temp[3:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        test_set.append([' '.join(temp), unit[-3:-2]])
    else:
        temp = temp[1:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        test_set.append([' '.join(temp), unit[-3:-2]])

print(train_set[0])
print(test_set[0])
print('\n')
print('All data sets are formatted!')

In [ ]:
print('Tokenizeing...')
print('\n')

def tokenize(data):
    res = []
    for samples in data:
        # nltk.word_tokenize用于取tokens
        temp_t = nltk.word_tokenize(samples[0])
        res.append([temp_t, samples[1]])
    return res


train_set = tokenize(train_set)
test_set = tokenize(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Tokenization completed!')

In [ ]:
print('Tokenizeing...')
print('\n')

def tokenize(data):
    res = []
    for samples in data:
        # nltk.word_tokenize用于取tokens
        temp_t = nltk.word_tokenize(samples[0])
        res.append([temp_t, samples[1]])
    return res


train_set = tokenize(train_set)
test_set = tokenize(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Tokenization completed!')

In [ ]:
print('Stemming...')
print('\n')
ps = PorterStemmer()


def Stemming(data):
    res = []
    stemmed_words_temp = []
    for units in data:
        for w in units[0]:
            #ps.stem(w):对当前单词w取stem word
            stemmed_words_temp.append(ps.stem(w))

        res.append([stemmed_words_temp, units[1]])
        stemmed_words_temp = []
    return res


train_set = Stemming(train_set)
test_set = Stemming(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Stemming completed!')

In [ ]:
print('Lemmatization...')
print('\n')
lem = WordNetLemmatizer()
stem = PorterStemmer()


def Lemmatization(data):
    lem_words_temp = []
    res = []
    for atoms in data:
        for w in atoms[0]:
            #lem.lemmatize(w, "v"): 对当前单词w取Lemmatization
            lem_words_temp.append(lem.lemmatize(w, "v"))
        res.append([lem_words_temp, atoms[1]])
        lem_words_temp = []

    return res


train_set = Lemmatization(train_set)
test_set = Lemmatization(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Lemmatization completed!')

In [ ]:
print('Corpus and labels gathering...')
print('\n')
corpus_test = []
corpus_train = []


def Gathering_corpus(data):
    res = []
    temp_res = []
    for aas in data:
        #corpus中仅仅需要第一个unit，即句子
        temp_res.append(aas[0])

    for aaas in temp_res:
        # 对于scikit-learn中的vectorizer，corpus必须为一句一句话的集合，而并非tokens。所以这里必须join然后加上句号。
        temp_ff = ' '.join(aaas)
        temp_ff = temp_ff + '.'
        res.append(temp_ff)
    return res


corpus_train = Gathering_corpus(train_set)
corpus_test = Gathering_corpus(test_set)
# train和test的corpus之和，成为corpus_sum
corpus_sum = []
corpus_sum = corpus_train
for things in corpus_test:
    corpus_sum.append(things)
print('len of corpus_sum:', len(corpus_sum))
print(corpus_sum[:5])
#取出label: Y_train and Y_test
Y_train = []
for thing in train_set:
    Y_train.append(thing[1])
Y_train = np.array(Y_train)
Y_test = []
for that in test_set:
    Y_test.append(that[1])
Y_test = np.array(Y_test)
print(Y_train[:5])
print(Y_test[:5])
print(Y_train.shape)
print(Y_test.shape)
print('\n')
print('Corpus and labels get!')

In [ ]:
print('tfidf processing...')
print('\n')
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(corpus_sum)
X_train_tfidf = X_tfidf[:16659]
X_test_tfidf = X_tfidf[16659:17469]
X_train_tfidf = X_train_tfidf.toarray()
X_test_tfidf = X_test_tfidf.toarray()
X_tfidf_arr = X_tfidf.toarray()
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(X_tfidf_arr.shape)
print('\n')
print('tfidf get!')

In [ ]:
'''Decision Tree'''
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print('Training GBC_model for TF-IDF...')

GBC_model = GradientBoostingClassifier(loss='deviance', n_estimators=300)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print('Fitting data in GBC_model...')
GBC_model.fit(X_train_tfidf, Y_train)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print('GBC_model model completed!')

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print('Calculating Y_hat_tfidf...')

Y_hat_tfidf = GBC_model.predict(X_test_tfidf)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print('Y_hat_tfidf get!')

In [ ]:
print('Scores of DT:\n')
print('F1 score:', f1_score(Y_test, Y_hat_tfidf, average='binary', pos_label='1'))
print('Precision:', precision_score(Y_test, Y_hat_tfidf, average='binary', pos_label='1'))
print('Recall:', recall_score(Y_test, Y_hat_tfidf, average='binary', pos_label='1'))
print('Accuracy:', accuracy_score(Y_test, Y_hat_tfidf))